# Stopping Time Analysis

Denote the space of suboptimal equilibria as $\pi_e$ and the globally optimal equilibrium as $\pi_g$. 

Define:
$\tau_e := min(t>0: \pi_t \in \pi_e)$\
$\tau_{e,i} := E[\tau_e | \pi_0 = i, \tau_e <\infty ]$

and similarly for $\tau_g, \tau_{g,i}$, where the indices $i$ denote a joint policy from an enumeration of the policy space. 

The below analysis is performed for the prisoner's dilemma problem with $\lambda^0 = 1/4, \lambda^1 = 3/4$

In [1]:
import numpy as np

from sympy import *
from sympy.printing.latex import LatexPrinter, print_latex
from sympy.interactive import printing
from IPython.display import Math, display

from tqdm import tqdm

The transition matrix; obtained via the BR graph and $\lambda$'s

In [2]:
# transition matrix
P_ = np.array([[1.    , 0.    , 0.    , 0.    ],
       [0.0625, 0.1875, 0.1875, 0.5625],
       [0.5625, 0.1875, 0.1875, 0.0625],
       [0.    , 0.    , 0.    , 1.    ]])
P = Matrix(P_).applyfunc(nsimplify)
P

Matrix([
[   1,    0,    0,    0],
[1/16, 3/16, 3/16, 9/16],
[9/16, 3/16, 3/16, 1/16],
[   0,    0,    0,    1]])

In [32]:
printing.default_latex(P)

'\\left[\\begin{matrix}1 & 0 & 0 & 0\\\\\\frac{1}{16} & \\frac{3}{16} & \\frac{3}{16} & \\frac{9}{16}\\\\\\frac{9}{16} & \\frac{3}{16} & \\frac{3}{16} & \\frac{1}{16}\\\\0 & 0 & 0 & 1\\end{matrix}\\right]'

In [3]:
# indices of suboptimal and optimal equilibria
pi_e = 0
pi_g = 3

mask_e = np.array([True if i!=pi_e else False for i in range(P_.shape[0])])
mask_g = np.array([True if i!=pi_g else False for i in range(P_.shape[0])])

In [4]:
# transition matrix removing global optimal equilibrium and suboptimal equilibrium respectively 
P_ng = Matrix(P_[:,mask_g][mask_g, :]/(1-P_[mask_g,pi_g, np.newaxis])).applyfunc(nsimplify)
P_ne = Matrix(P_[:,mask_e][mask_e, :]/(1-P_[mask_e,pi_e, np.newaxis])).applyfunc(nsimplify)

## First, calculate $\tau_e$

In [5]:
P_ng # the transition matrix given that global optimum is not visited

Matrix([
[  1,   0,   0],
[1/7, 3/7, 3/7],
[3/5, 1/5, 1/5]])

In [6]:
x_end = 0 # end state

tau_0, tau_1, tau_2 = symbols('tau0:3')
taus = [tau_0, tau_1, tau_2]

In [7]:
xs_ng = [0, 1, 2]

# define system of equations using conditional transition matrix above
eqs = [Eq(tau_0, 0),
        Eq(tau_1, 1 + sum([P_ng[1, x_1]*tau_i for x_1, tau_i in zip(xs_ng, taus)])),
        Eq(tau_2, 1 + sum(P_ng[2, x_1]*tau_i for x_1, tau_i in zip(xs_ng, taus)))]

The transition matrix gives us the following system of equations for hitting times corresponding to each initial state (policy):

In [8]:
# display system of equations
for eq in eqs:
    display(Math(printing.default_latex(eq)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

This system of equations admits the following solution:

In [9]:
solve(eqs)

{tau0: 0, tau1: 43/13, tau2: 27/13}

Now, we confirm the solution is correct via simulation

In [10]:
# simulation function to confirm answer
def sim_n(P, x_0, n_x, x_end):
    x_t = x_0
    t = 0
    
    while True:
        
        if x_t == x_end:
            return t
        
        while True:
            x_t_ = np.random.choice(range(P.shape[0]), p=P[x_t])
            if x_t_ != n_x:
                x_t = x_t_
                break
        t+= 1

In [11]:
# confirm answers above with simulation
n_samples = int(1e4)
x_0 = 2 # initial state (\tau_{x_0})
n_x = pi_g
x_end = pi_e
sims = np.array([sim_n(P_, x_0, n_x, x_end) for _ in tqdm(range(n_samples))])
sims = sims[sims!=-1]
np.average(sims) 

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4230.39it/s]


2.0805

In [12]:
27/13 # \tau_2

2.076923076923077

## Now, calculate $\tau_g$

In [13]:
P_ne # transition matrix conditioned on suboptimal equilibrium not being visited

Matrix([
[1/5, 1/5, 3/5],
[3/7, 3/7, 1/7],
[  0,   0,   1]])

In [14]:
# here, we calculate stopping time to the optimal equilibrium
x_end = 3 # end state

tau_1, tau_2, tau_3 = symbols('tau1:4')
taus = [tau_1, tau_2, tau_3]

In [15]:
xs_ne = [1, 2, 3]

eqs = [
        Eq(tau_3, 0),
        Eq(tau_2, 1 + sum(P_ne[2-1, x_1 - 1]*tau_i for x_1, tau_i in zip(xs_ne, taus))),
        Eq(tau_1, 1 + sum([P_ne[1-1, x_1 - 1]*tau_i for x_1, tau_i in zip(xs_ne, taus)]))
      ]

The transition matrix gives us the following system of equations for hitting times corresponding to each initial state (policy)

In [16]:
# display system of equations
for eq in eqs:
    display(Math(printing.default_latex(eq)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The system of equations admits the following solution:

In [17]:
# solve system of equations
solve(eqs)

{tau1: 27/13, tau2: 43/13, tau3: 0}

Now, we confirm the solution is correct via simulation:

In [18]:
# confirm with simulation
n_samples = int(1e4)
x_0 = 1
n_x = pi_e
x_end = pi_g
sims = np.array([sim_n(P_, x_0, n_x, x_end) for _ in tqdm(range(n_samples))])
sims = sims[sims!=-1]
np.average(sims)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4661.33it/s]


2.0574

In [19]:
27/13

2.076923076923077

##  $\min(\tau_e, \tau_g)$

In [20]:
P

Matrix([
[   1,    0,    0,    0],
[1/16, 3/16, 3/16, 9/16],
[9/16, 3/16, 3/16, 1/16],
[   0,    0,    0,    1]])

In [21]:
# here, we calculate stopping time to the optimal equilibrium
x_end = [0, 3] # end state

tau_0, tau_1, tau_2, tau_3 = symbols('tau0:4')
taus = [tau_0, tau_1, tau_2, tau_3]

In [22]:
xs = [0, 1, 2, 3]

eqs = [
        Eq(tau_3, 0),
        Eq(tau_2, 1 + sum(P[2, x_1]*tau_i for x_1, tau_i in zip(xs, taus))),
        Eq(tau_1, 1 + sum([P[1, x_1]*tau_i for x_1, tau_i in zip(xs, taus)])),
        Eq(tau_0, 0)
      ]

In [23]:
# display system of equations
for eq in eqs:
    display(Math(printing.default_latex(eq)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [24]:
solve(eqs)

{tau0: 0, tau1: 8/5, tau2: 8/5, tau3: 0}

In [25]:
# confirm via simulation
# simulation function to confirm answer
def sim_n2(P, x_0, x_end):
    x_t = x_0
    t = 0
    
    while True:
        if x_t in x_end:
            return [x_t, t]
        
        x_t = np.random.choice(range(P.shape[0]), p=P[x_t])
        t+= 1

In [26]:
# confirm with simulation
n_samples = int(1e5)
x_0 = 1
sims = np.array([sim_n2(P_, x_0, x_end) for _ in tqdm(range(n_samples))])
np.average(sims[:,1])

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:13<00:00, 7685.51it/s]


1.59509

In [29]:
filt_g = sims[:, 0] == pi_g
sims_g = sims[filt_g][:,1]
print(np.average(sims_g))

filt_e = sims[:,0] == pi_e
sims_e = sims[filt_e][:,1]
print(np.average(sims_e))

1.3948261578342553
2.1907241077815756


## Now, Calculate the probability of ending at each equilibrium: $P(\tau_e < \tau_g)$


$P(\tau_e < \tau_g) = \frac{E[\min(\tau_e,\tau_g)] - E[\tau_g | \tau_g<\tau_e]}{E[\tau_e | \tau_e < \tau_g] - E[\tau_g | \tau_g < \tau_e]}$

In [30]:
x_0 = 1
tau_min = 8/5
tau_e1 = np.average(sims_e)#43/13
tau_g1 = np.average(sims_g)#27/13

In [31]:
(tau_min - tau_g1) / (tau_e1 - tau_g1)

0.25778913261345254